In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [2]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aditg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aditg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aditg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
brand_category_df = pd.read_csv("C://Users//aditg//Downloads//DS_NLP_search_data//brand_category.csv")
categories_df = pd.read_csv("C://Users//aditg//Downloads//DS_NLP_search_data//categories.csv")
offer_retailer_df = pd.read_csv("C://Users//aditg//Downloads//DS_NLP_search_data//offer_retailer.csv")

## EDA

In [4]:
brand_category_df.head()

,BRAND,BRAND_BELONGS_TO_CATEGORY,RECEIPTS
0,CASEYS GEN STORE,Tobacco Products,2950931
1,CASEYS GEN STORE,Mature,2859240
2,EQUATE,Hair Removal,893268
3,PALMOLIVE,Bath & Body,542562
4,DAWN,Bath & Body,301844


In [5]:
brand_category_df.shape

(9906, 3)

In [6]:
categories_df.head()

,CATEGORY_ID,PRODUCT_CATEGORY,IS_CHILD_CATEGORY_TO
0,1f7d2fa7-a1d7-4969-aaf4-1244f232c175,Red Pasta Sauce,Pasta Sauce
1,3e48a9b3-1ab2-4f2d-867d-4a30828afeab,Alfredo & White Pasta Sauce,Pasta Sauce
2,09f3decc-aa93-460d-936c-0ddf06b055a3,Cooking & Baking,Pantry
3,12a89b18-4c01-4048-94b2-0705e0a45f6b,Packaged Seafood,Pantry
4,2caa015a-ca32-4456-a086-621446238783,Feminine Hygeine,Health & Wellness


In [7]:
categories_df.shape

(118, 3)

In [8]:
offer_retailer_df.head()

,OFFER,RETAILER,BRAND
0,Spend $50 on a Full-Priced new Club Membership,SAMS CLUB,SAMS CLUB
1,"Beyond Meat® Plant-Based products, spend $25",NaN,BEYOND MEAT
2,Good Humor Viennetta Frozen Vanilla Cake,NaN,GOOD HUMOR
3,"Butterball, select varieties, spend $10 at Dil...",DILLONS FOOD STORE,BUTTERBALL
4,"GATORADE® Fast Twitch®, 12-ounce 12 pack, at A...",AMAZON,GATORADE


In [9]:
offer_retailer_df.shape

(384, 3)

In [10]:
brand_category_df.isna().sum()

BRAND                        1
BRAND_BELONGS_TO_CATEGORY    0
RECEIPTS                     0
dtype: int64

In [11]:
categories_df.isna().sum()

CATEGORY_ID             0
PRODUCT_CATEGORY        0
IS_CHILD_CATEGORY_TO    0
dtype: int64

In [12]:
offer_retailer_df.isna().sum()

OFFER         0
RETAILER    146
BRAND         0
dtype: int64

In [13]:
brand_category_df.dropna(inplace=True)
brand_category_df.reset_index(drop=True, inplace=True)

offer_retailer_df.fillna("No Retailer Information", inplace=True)

In [14]:
brand_belongs_to_category = brand_category_df['BRAND_BELONGS_TO_CATEGORY'].unique()
prod_category = categories_df['PRODUCT_CATEGORY'].unique()

In [15]:
flag = 0
for i in range(len(prod_category)):
    if prod_category[i] not in brand_belongs_to_category:
        print(prod_category[i])
        flag = 1
if flag == 0:
    print("PRODUCT_CATEGORY and BRAND_BELONGS_TO_CATEGORY contain exactly the same values")

PRODUCT_CATEGORY and BRAND_BELONGS_TO_CATEGORY contain exactly the same values


In [16]:
# Merging brand_category and categories
brand_category_df.rename(columns={'BRAND_BELONGS_TO_CATEGORY': "CATEGORY"}, inplace=True)
categories_df.rename(columns={'PRODUCT_CATEGORY': "CATEGORY"}, inplace=True)

brand_categories_df = brand_category_df.merge(categories_df, on = 'CATEGORY', how = 'left')

In [17]:
brand_categories_df.head()

,BRAND,CATEGORY,RECEIPTS,CATEGORY_ID,IS_CHILD_CATEGORY_TO
0,CASEYS GEN STORE,Tobacco Products,2950931,4442fa51-d4b7-4d56-ad7c-d18cfd2aff84,Mature
1,CASEYS GEN STORE,Mature,2859240,f81d7064-52da-4bd7-98eb-7a14be267720,Mature
2,EQUATE,Hair Removal,893268,539953fe-50bf-4a00-8a21-c64752d6ed7e,Health & Wellness
3,PALMOLIVE,Bath & Body,542562,47fae368-9b64-40be-82c9-898f953e9d66,Health & Wellness
4,DAWN,Bath & Body,301844,47fae368-9b64-40be-82c9-898f953e9d66,Health & Wellness


In [18]:
brand_categories_df.shape

(9905, 5)

In [19]:
duplicate = brand_categories_df[brand_categories_df.duplicated()]

In [20]:
duplicate

,BRAND,CATEGORY,RECEIPTS,CATEGORY_ID,IS_CHILD_CATEGORY_TO


## Implementing Elastic Search

In [21]:
es = Elasticsearch("http://localhost:9200")

In [44]:
mappings = {
        "properties":
        {
            "BRAND": {"type": "text", "analyzer": "standard"},
            "CATEGORY": {"type": "text", "analyzer": "english"},
            "IS_CHILD_CATEGORY_TO": {"type": "text", "analyzer": "english"}
        }
    }

es.indices.create(index="products", mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'})

In [75]:
mappings = {
    "properties":
    {
        "OFFER": {"type": "text", "analyzer": "english"},
        "RETAILER": {"type": "text", "analyzer": "standard"},
        "BRAND": {"type": "text", "analyzer": "standard"}
    }
}

es.indices.create(index="offers", mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'offers'})

In [45]:
for i, row in brand_categories_df.iterrows():
    doc = {
        "BRAND": row["BRAND"],
        "CATEGORY": row["CATEGORY"],
        "IS_CHILD_CATEGORY_TO": row["IS_CHILD_CATEGORY_TO"],
    }
            
    es.index(index="products", id=i, document=doc)

es.indices.refresh(index="products")
es.cat.count(index="products", format="json")

In [76]:
for i, row in offer_retailer_df.iterrows():
    doc = {
        "OFFER": row["OFFER"],
        "RETAILER": row["RETAILER"],
        "BRAND": row["BRAND"]
    }
    
    es.index(index="offers", id=i, document=doc)
    
es.indices.refresh(index="offers")
es.cat.count(index="offers", format="json")

In [22]:
search_query = input("Enter search query: ")

Enter search query: gatorade


In [23]:
def preprocess_query_nltk(query):
    words = word_tokenize(query)
    words = [word for word in words if word not in string.punctuation]
    
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    processed_query = ' '.join(words)
    
    return processed_query

search_query_best = preprocess_query_nltk(search_query)
words = search_query_best.split()
search_query_prefix = '* '.join(words) + "*"

In [24]:
resp = es.search(
    index="offers",
    query={
            "bool": 
            {
              "should": 
              [
                  {
                    "multi_match": 
                      {
                        "query": search_query_best,
                        "fields": ["OFFER", "RETAILER", "BRAND"],
                        "type": "best_fields"
                      }
                  },
                  {
                    "multi_match": 
                      {
                        "query": search_query_prefix,
                        "fields": ["OFFER", "RETAILER", "BRAND"],
                        "type": "phrase_prefix"
                      }
                  }
              ]
            }
          }          
    )
resp.body

{'took': 93,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 7, 'relation': 'eq'},
  'max_score': 9.683443,
  'hits': [{'_index': 'offers',
    '_id': '4',
    '_score': 9.683443,
    '_source': {'OFFER': 'GATORADE® Fast Twitch®, 12-ounce 12 pack, at Amazon Storefront*',
     'RETAILER': 'AMAZON',
     'BRAND': 'GATORADE'}},
   {'_index': 'offers',
    '_id': '16',
    '_score': 9.683443,
    '_source': {'OFFER': 'GATORLYTE® OR GATORADE™ Fit Single Serve Bottle, select varieties, buy 2',
     'RETAILER': 'No Retailer Information',
     'BRAND': 'GATORADE'}},
   {'_index': 'offers',
    '_id': '190',
    '_score': 9.683443,
    '_source': {'OFFER': 'GATORADE® Fast Twitch®, 12-ounce single serve, buy 2 at Kroger',
     'RETAILER': 'FRED MEYER',
     'BRAND': 'GATORADE'}},
   {'_index': 'offers',
    '_id': '302',
    '_score': 9.683443,
    '_source': {'OFFER': 'GATORADE® Fast Twitch®, 12-ounce single serve',
     'R

In [25]:
valid_entries = []
for entry in resp['hits']['hits']:
    brand = entry['_source']['BRAND']
    resp1 = es.search(
        index="products",
        query={
                "bool": 
                {
                    "must": 
                    {
                        "match_phrase": 
                        {
                            "BRAND": brand
                        }
                    }
                }
            }            
    )
    if resp1['hits']['total']['value'] > 0:
        valid_entries.append(entry)

In [26]:
valid_entries

[{'_index': 'offers',
  '_id': '4',
  '_score': 9.683443,
  '_source': {'OFFER': 'GATORADE® Fast Twitch®, 12-ounce 12 pack, at Amazon Storefront*',
   'RETAILER': 'AMAZON',
   'BRAND': 'GATORADE'}},
 {'_index': 'offers',
  '_id': '16',
  '_score': 9.683443,
  '_source': {'OFFER': 'GATORLYTE® OR GATORADE™ Fit Single Serve Bottle, select varieties, buy 2',
   'RETAILER': 'No Retailer Information',
   'BRAND': 'GATORADE'}},
 {'_index': 'offers',
  '_id': '190',
  '_score': 9.683443,
  '_source': {'OFFER': 'GATORADE® Fast Twitch®, 12-ounce single serve, buy 2 at Kroger',
   'RETAILER': 'FRED MEYER',
   'BRAND': 'GATORADE'}},
 {'_index': 'offers',
  '_id': '302',
  '_score': 9.683443,
  '_source': {'OFFER': 'GATORADE® Fast Twitch®, 12-ounce single serve',
   'RETAILER': 'No Retailer Information',
   'BRAND': 'GATORADE'}},
 {'_index': 'offers',
  '_id': '307',
  '_score': 9.683443,
  '_source': {'OFFER': 'GATORADE® Fast Twitch®, 12-ounce single serve, at Kroger',
   'RETAILER': 'DILLONS GROC